## Loading Libraries

In [1]:
import pandas as pd
import numpy as np
import torch
import evaluate

c:\Users\Pawel\anaconda3\envs\env_torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda')

## Loading Dataset

In [3]:
df = pd.read_csv('../data/processed/train_sample_processed.csv')

In [4]:
statuses = np.unique(df['OpenStatus'].values)

id2label = {idx: label for idx, label in enumerate(statuses)}
label2id = {label: idx for idx, label in enumerate(statuses)}

In [5]:

from custom_dataset import GithubDataset
from torch.utils.data import random_split
from datasets import Dataset

# Create the pytorch dataset
full_dataset = GithubDataset(df)

In [6]:
train_dataset, validation_dataset, test_dataset = random_split(full_dataset, [0.7, 0.25, 0.05])

train_dataset = Dataset.from_dict(train_dataset[:])
validation_dataset = Dataset.from_dict(validation_dataset[:])
test_dataset = Dataset.from_dict(test_dataset[:])

In [7]:
train_dataset = train_dataset.rename_columns({"text_content": "text", "status": "label"})
validation_dataset = validation_dataset.rename_columns({"text_content": "text", "status": "label"})
test_dataset = test_dataset.rename_columns({"text_content": "text", "status": "label"})

columns_to_remove = [
  'tags_onehot',
  'unrecognized_tags_count',
  'reputation',
  'undeleted_answers',
  'user_life_days',
  'title'
]

train_dataset = train_dataset.remove_columns(columns_to_remove)
validation_dataset = validation_dataset.remove_columns(columns_to_remove)
test_dataset = test_dataset.remove_columns(columns_to_remove)

### Tokenizing Data

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [9]:
MAX_TEXT_CONTENT = 128

def tokenize_func(batch):
  tokenized_batch = tokenizer(batch['text'], padding=True, truncation=True, max_length=MAX_TEXT_CONTENT, return_tensors="pt")
  tokenized_batch["labels"] = [label2id[label] for label in batch["label"]]
  return tokenized_batch

In [10]:
tokenized_train_dataset = train_dataset.map(tokenize_func, batched=True)
tokenized_validation_dataset = validation_dataset.map(tokenize_func, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_func, batched=True)

Map: 100%|██████████| 7013/7013 [00:01<00:00, 6114.92 examples/s]


#### COMMENT - IDEAS

We probably should:
- retrain the whole model (probably smaller) with
- better tokenizer - built up from the ground including all the names of the specific tech (languages, frameworks, IDEs, etc.)

## Model - Custom

In [11]:
from custom_model import AutoCompositeModel

model = AutoCompositeModel(device).to(device)

c:\Users\Pawel\anaconda3\envs\env_torch\lib\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [12]:
model.my_new_layers[0].weight.dtype

torch.float32

In [13]:
input = torch.tensor(tokenized_test_dataset['input_ids'][:10]).float().to(device)
input.dtype

torch.float32

## Training Setup

### Data Loaders

In [14]:
from torch.utils.data import DataLoader
training_loader = DataLoader(tokenized_train_dataset, batch_size=16, shuffle=True)
validation_loader = DataLoader(tokenized_validation_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(tokenized_test_dataset, batch_size=16, shuffle=True)

### Loss Function

In [15]:
loss_fn = torch.nn.CrossEntropyLoss()

### Optimizer

In [16]:
# Optimizers specified in the torch.optim package
# optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, weight_decay=1e-4)

In [17]:
from torchmetrics import Accuracy

accuracy_metric = Accuracy(task='multiclass', num_classes=5).to(device)

### Trainer Setup

In [18]:
from training_own import Trainer, TrainerConfiguration, get_model_params

config = TrainerConfiguration(
  training_loader=training_loader,
  validation_loader=validation_loader,
  optimizer=optimizer,
  loss_fn=loss_fn,
  accuracy_metric=accuracy_metric,
  device=device
)

In [19]:
get_model_params(model.model_content)

3558144

In [20]:
trainer = Trainer(model=model, trainer_configuration=config, input_column='input_ids', output_column='labels')

In [21]:
# for i, data in enumerate(training_loader):
#   if i > 0: break
#   # print(i, torch.FloatTensor(data['input_ids']))
  
#   # a = [torch.FloatTensor([1]).view(1, -1), torch.FloatTensor([2]).view(1, -1)]
#   stacked = torch.stack(data['input_ids']).long()
#   print(stacked.dtype)

#   # model(data['input_ids'])
  

In [22]:
len(training_loader)

6137

In [23]:
trainer.train_one_epoch(logging_frequency=100, evaluate_when_logging=False)
trainer.train_one_epoch(logging_frequency=1000, evaluate_when_logging=True)
# trainer.train_one_epoch(logging_frequency=100, evaluate_when_logging=False)
# trainer.train_one_epoch(logging_frequency=100, evaluate_when_logging=False)
# trainer.train_one_epoch(logging_frequency=100, evaluate_when_logging=False)

 batch 100 training_loss: 1.6380150377750398 training_accuracy: 0.0768750011920929
 batch 200 training_loss: 1.640323760509491 training_accuracy: 0.06187499687075615
 batch 300 training_loss: 1.6398781383037566 training_accuracy: 0.07187499850988388
 batch 400 training_loss: 1.6423733866214751 training_accuracy: 0.07000000029802322
 batch 500 training_loss: 1.6402418160438537 training_accuracy: 0.08562500029802322
 batch 600 training_loss: 1.638577687740326 training_accuracy: 0.07187499850988388
 batch 700 training_loss: 1.6382822072505951 training_accuracy: 0.0793749988079071
 batch 800 training_loss: 1.6397054278850556 training_accuracy: 0.07625000178813934
 batch 900 training_loss: 1.6389861512184143 training_accuracy: 0.08062499761581421
 batch 1000 training_loss: 1.6367674934864045 training_accuracy: 0.08999999612569809
 batch 1100 training_loss: 1.6375688672065736 training_accuracy: 0.08812499791383743
 batch 1200 training_loss: 1.6333950936794281 training_accuracy: 0.08249999582

(1.5921369595527648, tensor(0.2372, device='cuda:0'))